In [109]:
import sklearn
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.metrics import  accuracy_score,log_loss, r2_score, roc_auc_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

from sklearn.tree import plot_tree
import matplotlib.pyplot as plt
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.preprocessing import OneHotEncoder


import warnings
warnings.filterwarnings("ignore")


In [110]:
df = pd.read_csv(r'C:\Users\DAI.STUDENTSDC\Desktop\Machine Learning\Assignment\restaurant_customer_satisfaction.csv')
df.head()

,CustomerID,Age,Gender,Income,VisitFrequency,AverageSpend,PreferredCuisine,TimeOfVisit,GroupSize,DiningOccasion,MealType,OnlineReservation,DeliveryOrder,LoyaltyProgramMember,WaitTime,ServiceRating,FoodRating,AmbianceRating,HighSatisfaction
0,654,35,Male,83380,Weekly,27.829142,Chinese,Breakfast,3,Business,Takeaway,0,1,1,43.523929,2,5,4,0
1,655,19,Male,43623,Rarely,115.408622,American,Dinner,1,Casual,Dine-in,0,0,0,57.524294,5,5,3,0
2,656,41,Female,83737,Weekly,106.693771,American,Dinner,6,Celebration,Dine-in,0,1,0,48.682623,3,4,5,0
3,657,43,Male,96768,Rarely,43.508508,Indian,Lunch,1,Celebration,Dine-in,0,0,0,7.552993,4,5,1,0
4,658,55,Female,67937,Monthly,148.084627,Chinese,Breakfast,1,Business,Takeaway,0,0,1,37.789041,2,3,5,0


## 1. EDA

a.	Is Age correlated significantly with Average Spend?

In [111]:
df['Age'].corr(df['AverageSpend']) #A very weak or negligible linear relationship between the two variables.

0.017765408060572693

b.	Calculate the mean average spending for both the genders separately.

In [112]:
df.groupby('Gender')['AverageSpend'].mean()

Gender
Female    105.471643
Male      105.850916
Name: AverageSpend, dtype: float64

c.	Calculate the mean of Age for all the categories of Time of Visit separately.

In [113]:
df.groupby('TimeOfVisit')['Age'].mean()

TimeOfVisit
Breakfast    43.559289
Dinner       43.500000
Lunch        44.432271
Name: Age, dtype: float64

d.	Among the people rarely visiting, what is the most preferred cuisine by percentage?

In [114]:
df.columns

Index(['CustomerID', 'Age', 'Gender', 'Income', 'VisitFrequency',
       'AverageSpend', 'PreferredCuisine', 'TimeOfVisit', 'GroupSize',
       'DiningOccasion', 'MealType', 'OnlineReservation', 'DeliveryOrder',
       'LoyaltyProgramMember', 'WaitTime', 'ServiceRating', 'FoodRating',
       'AmbianceRating', 'HighSatisfaction'],
      dtype='object')

In [115]:
rare_visitors = df[df['VisitFrequency'] == 'Rarely']
total = rare_visitors.groupby('PreferredCuisine')['CustomerID'].count().sum()
cuisine_percentage = rare_visitors.groupby('PreferredCuisine')['CustomerID'].count() / total * 100

cuisine_percentage = cuisine_percentage.reset_index()
cuisine_percentage.columns = ['PreferredCuisine', 'Customer Percentage']

print(cuisine_percentage)

  PreferredCuisine  Customer Percentage
0         American            16.613419
1          Chinese            18.530351
2           Indian            18.210863
3          Italian            29.073482
4          Mexican            17.571885


e.	Among the people weekly visiting, what is the most frequent meal type?

In [116]:
weekly_visitors = df[df['VisitFrequency'] == 'Weekly']
weekly_visitors['MealType'].value_counts()

MealType
Takeaway    307
Dine-in     299
Name: count, dtype: int64

f.	Among the business type of dining occasion, what is most frequent preferred cuisine?

In [117]:
df[df['DiningOccasion'] == 'Business']['PreferredCuisine'].value_counts()

PreferredCuisine
Italian     125
Chinese     104
Indian       95
Mexican      89
American     87
Name: count, dtype: int64

g.	With the group size bigger than 5, what is percentage of business type of dining occasion?

In [118]:
occasion = df[df['GroupSize'] > 5]['DiningOccasion'].value_counts()
total = occasion.sum()
percentage = occasion / total * 100
percentage.reset_index()
# percentage.columns = ['DiningOccasion', 'Percentage']
print(percentage)


DiningOccasion
Celebration    35.103245
Business       33.333333
Casual         31.563422
Name: count, dtype: float64


h.	Among the customers preferring Chinese cuisine, how much percentage are satisfied?

In [119]:
c_hs = df[df['PreferredCuisine'] == 'Chinese']['HighSatisfaction'].value_counts()
c_hs

HighSatisfaction
0    268
1     42
Name: count, dtype: int64

In [120]:
total = c_hs.sum()
percentage = c_hs[1] / total * 100
percentage

13.548387096774196

i.	What is average rating given to the food by the customers preferring Chinese cuisine?

In [121]:
df[df['PreferredCuisine'] == 'Chinese']['FoodRating'].mean()

3.0709677419354837

j.	For the customers preferring Chinese cuisine, how much is the average waiting time?

In [122]:
df[df['PreferredCuisine']=='Chinese']['WaitTime'].mean()

29.51653917994555

## 2.	Do the following considering encoded HighSatisfaction as response(dependent) variable.

In [123]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.preprocessing import OneHotEncoder


import warnings
warnings.filterwarnings("ignore")

In [124]:
df = pd.read_csv(r'C:\Users\DAI.STUDENTSDC\Desktop\Machine Learning\Assignment\restaurant_customer_satisfaction.csv')


X = df.drop(columns=['HighSatisfaction'], axis=1)
y = df['HighSatisfaction']

In [125]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   CustomerID            1500 non-null   int64  
 1   Age                   1500 non-null   int64  
 2   Gender                1500 non-null   object 
 3   Income                1500 non-null   int64  
 4   VisitFrequency        1500 non-null   object 
 5   AverageSpend          1500 non-null   float64
 6   PreferredCuisine      1500 non-null   object 
 7   TimeOfVisit           1500 non-null   object 
 8   GroupSize             1500 non-null   int64  
 9   DiningOccasion        1500 non-null   object 
 10  MealType              1500 non-null   object 
 11  OnlineReservation     1500 non-null   int64  
 12  DeliveryOrder         1500 non-null   int64  
 13  LoyaltyProgramMember  1500 non-null   int64  
 14  WaitTime              1500 non-null   float64
 15  ServiceRating        

In [126]:
#Checking for null values
df.isna().sum()

CustomerID              0
Age                     0
Gender                  0
Income                  0
VisitFrequency          0
AverageSpend            0
PreferredCuisine        0
TimeOfVisit             0
GroupSize               0
DiningOccasion          0
MealType                0
OnlineReservation       0
DeliveryOrder           0
LoyaltyProgramMember    0
WaitTime                0
ServiceRating           0
FoodRating              0
AmbianceRating          0
HighSatisfaction        0
dtype: int64

**A.** Do the hot encoding / dummying of all the non-numeric variables

**B.** Considering the Grid Search CV / Randomized Search CV with parameters of your choice, test the accuracy on following models:
* SVC (kernel=’linear’)
* SVC (kernel=’rbf’)
* Decision Tree Classifier
* Random Forest Classifier

In [127]:
svc_l = SVC(kernel='linear', random_state=24)
svc_r = SVC(kernel='rbf', random_state=24)
dtc = DecisionTreeClassifier(random_state=24)
rfc = RandomForestClassifier(random_state=24)

In [128]:
#Creating a list of models and the params to be used for each model

models=[
    {   
        'name': 'Decision Tree Classifier',
        'model' : dtc,
        'params' : {
            'model__criterion' : ['gini', 'entropy'],
            'model__min_samples_split' : np.arange(2, 10),
            'model__min_samples_leaf' : [1, 10],
            'model__max_depth' : [None, 4, 6]
        }
    },
    {
        'name': 'Random Forest Classifier',
        'model' : rfc,
        'params' : {
            'model__n_estimators' : [10, 50, 100],
            'model__criterion' : ['gini', 'entropy'],
            'model__max_depth' : [1, 3, 5]
        }
    },
    {   
        'name': 'Support Vector Machine - Linear',
        'model' : svc_l,
        'params' : {
            'model__decision_function_shape' : ['ovo', 'ovr'],
            'model__degree' : np.arange(3, 10),
            'model__max_iter' : np.arange(5),
            'model__gamma' : ['scale', 'auto']
        }
    },
    {
        'name': 'Support Vector Machine - RBF',
        'model' : svc_r,
        'params' : {
            'model__decision_function_shape' : ['ovo', 'ovr'],
            'model__degree' : np.arange(3, 10),
            'model__max_iter' : np.arange(5),
            'model__gamma' : ['scale', 'auto']
        }
    },
]

In [129]:
# Creating the one hot encoder and column transformer 


ohe = OneHotEncoder(
    sparse_output=False, 
    drop='first'
).set_output(transform='pandas')


ct = make_column_transformer(
    ('passthrough', make_column_selector(dtype_exclude=[
        'object',
        'category',
    ])),
    (ohe, make_column_selector(dtype_include=[
        'object',
        'category',
    ])),
    verbose_feature_names_out=False,
).set_output(transform='pandas')

In [130]:
#Loop for running GridsearchCV for each model
for model in models:
    pipe = Pipeline([
        ('column_transformer', ct),
        ('model', model['model']),
    ])

    
    kfold = StratifiedKFold(
        n_splits=5,
        random_state=24, 
        shuffle=True
    )
    
    gcv = GridSearchCV(
        estimator=pipe, 
        param_grid=model['params'], 
        cv=kfold, 
        scoring='accuracy',
    ) 

    gcv.fit(X, y)
    

    print(model['name'])
    print("Accuracy: ", gcv.best_score_)
    print("Prameters: ", gcv.best_params_)
    print()


Decision Tree Classifier
Accuracy:  0.8673333333333334
Prameters:  {'model__criterion': 'entropy', 'model__max_depth': 6, 'model__min_samples_leaf': 1, 'model__min_samples_split': 7}

Random Forest Classifier
Accuracy:  0.8686666666666667
Prameters:  {'model__criterion': 'entropy', 'model__max_depth': 5, 'model__n_estimators': 10}

Support Vector Machine - Linear
Accuracy:  0.5966666666666667
Prameters:  {'model__decision_function_shape': 'ovo', 'model__degree': 3, 'model__gamma': 'scale', 'model__max_iter': 3}

Support Vector Machine - RBF
Accuracy:  0.5653333333333334
Prameters:  {'model__decision_function_shape': 'ovo', 'model__degree': 3, 'model__gamma': 'scale', 'model__max_iter': 1}



**C.** Mention for which model you are getting lowest log loss score

In [131]:
# Creating the one hot encoder and column transformer 

ohe = OneHotEncoder(
    sparse_output=False, 
    drop='first'
).set_output(transform='pandas')


ct = make_column_transformer(
    ('passthrough', make_column_selector(dtype_exclude=[
        'object',
        'category',
    ])),
    (ohe, make_column_selector(dtype_include=[
        'object',
        'category',
    ])),
    verbose_feature_names_out=False,
).set_output(transform='pandas')

In [132]:
#Creating estimators for the pipeline
svc = SVC(random_state=24)
dtc = DecisionTreeClassifier(random_state=24)
rfc = RandomForestClassifier(random_state=24, )
 

kfold = StratifiedKFold(n_splits=5,random_state=24, shuffle=True)

In [133]:
# Creating tnhe pipeline with empty model
pipe = Pipeline([
    ('column_transformer', ct),
    ('model', None),
])


In [134]:
#List of paramaters to be tuned

params = [
    {
        'model' : [svc],
        'model__kernel': ['linear', 'rbf'],
        'model__decision_function_shape':['ovo', 'ovr'],
        'model__degree': np.arange(3, 10),
        'model__max_iter': np.arange(5),
        'model__gamma': ['scale', 'auto'],
    },
    {
        'model' : [dtc],
        'model__criterion': ['gini', 'entropy'],
        'model__min_samples_split':np.arange(1, 10),
        'model__min_samples_leaf':(1, 10),
        'model__max_depth':[None,4,6],
    },
    {
        'model' : [rfc],
        'model__n_estimators': [10,50,100],
        'model__criterion': ['gini', 'entropy'],
        'model__max_depth': [1, 3, 5],
    }
]



In [135]:
# Defininf the grid search
gcv = GridSearchCV(
    estimator=pipe, 
    param_grid=params, 
    cv=kfold, 
    scoring='neg_log_loss', 
)

# Fitting the model
gcv.fit(X, y)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=24, shuffle=True),
             estimator=Pipeline(steps=[('column_transformer',
                                        ColumnTransformer(transformers=[('passthrough',
                                                                         'passthrough',
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x0000026108E25820>),
                                                                        ('onehotencoder',
                                                                         OneHotEncoder(drop='first',
                                                                                       sparse_output=False),
                                                                         <sklearn.compose....
                         {'model': [DecisionTreeClassifier(random_state=24)],
                          'model__criterion': ['gini', 'entropy'],
                          'model__max_depth': [None, 4, 6],
                          'model__min_samples_leaf': (1, 10),
                          'model__min_samples_split': array([1, 2, 3, 4, 5, 6, 7, 8, 9])},
                         {'model': [RandomForestClassifier(random_state=24)],
                          'model__criterion': ['gini', 'entropy'],
                          'model__max_depth': [1, 3, 5],
                          'model__n_estimators': [10, 50, 100]}],
             scoring='neg_log_loss')

In [136]:
# Print the best model and its log loss error
print('Best Model')
print('Best score:', gcv.best_score_)
print('Best params_:', gcv.best_params_ )

Best Model
Best score: -0.29059112611523336
Best params_: {'model': RandomForestClassifier(random_state=24), 'model__criterion': 'entropy', 'model__max_depth': 5, 'model__n_estimators': 100}
